In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib import ticker

## 01. Parameters
### 01-01. Physical parameters
#### - Computational domain: 2D square region

In [ ]:
# g?
g = 9.81

# rho0
rho0 = 1e-2

# Temporal
t_sta = 0.
t_end = 150.

# Spatial
x_min = -10.
x_max = 10.

y_min = copy.copy(x_min) # shallow copy
y_max = copy.copy(x_max)

### 01-02. Computational parameters
#### - Sizes of timestep and grids
#### - Sampling frequency 

In [ ]:
# Sampling
sample_freq = 10 # sampling data per 10 timesteps

# Temporal
dt = 5e-2
nt = int((t_end - t_sta) / dt) + 1

# Spatial
nx = 128
ny = copy.copy(nx)

dx = np.abs(x_max - x_min) / nx
dy = np.abs(y_max - y_min) / ny

### 01-03. Allocation of memory on each variable (n, u1, u2) and mesh (X, Y) 

In [ ]:
# Variables
#eta  = np.zeros(shape=(2, nx, ny)) #(time(old(0), new(1)), x, y)
#velx = copy.deepcopy(eta) # deep copy
#vely = copy.deepcopy(eta)
eta  = np.zeros(shape=(3, nx, ny)) #(time(old(0), new(1), half(2)), x, y) for Leapfrog integration
velx = copy.deepcopy(eta) # deep copy
vely = copy.deepcopy(eta)

# Grids
x = np.linspace(x_min, x_max, nx)
y = copy.copy(x) # shallow copy
X, Y = np.meshgrid(x, y) # For visualization

## 02. Time integration
### 02-01. Initial condition

In [ ]:
# Continuous I.C.
eta[0,:,:] = rho0 + 0.001 * np.exp(-0.5 * (X**2 + Y**2))

## Discontinuous I.C.
#for i in range(0, nx):
#    for j in range(0, ny):
#        eta[0, i, j] = np.where((X[i, j]**2 + Y[i, j]**2) < (0.2*x_max)**2, rho0*1.1, rho0)

velx[0,:,:] = 0.
vely[0,:,:] = 0.

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X, Y, eta[0, :, :], cmap=cm.coolwarm,
                       vmin=(1.0-0.08)*rho0, vmax=(1.0+0.08)*rho0, 
                       linewidth=0, antialiased=False)

# ax.view_init(elev=10,azim=10)

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim((1.0-0.08)*rho0, (1.0+0.08)*rho0)

formatter = ticker.ScalarFormatter(useMathText=True)
# Formatting of scientific representation
formatter.set_scientific(True) 
# Formatting of floating point representation
formatter.set_powerlimits((-2,2)) 
formatter.set_scientific
ax.zaxis.set_major_formatter(formatter)

# Make the panes transparent(1)
# ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

# # Make the panes transparent(2)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
# ## Set color to white
# ax.xaxis.pane.set_edgecolor('w')
# ax.yaxis.pane.set_edgecolor('w')
# ax.zaxis.pane.set_edgecolor('w')

# Make the grid lines transparent(1)
# ax.xaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)
# ax.yaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)
# ax.zaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)

# Make the grid lines transparent(2): get rid of grid
# ax.grid(False)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=15, pad=0.15, extend='both')

ax.set_title('Elapsed Time: ' + str(0) + '[s]', loc='left')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')

# Save figure
id_snapshot = 0
fname = 'central2d/img_finite_media/leapfrog/continuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
#fname = 'central2d/img_finite_media/leapfrog/discontinuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
##fname = 'central2d/img_finite_media/euler/continuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
#fname = 'central2d/img_finite_media/euler/discontinuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
plt.savefig(fname, format='tiff', bbox_inches='tight', dpi=200)
id_snapshot += 1

plt.show()

### 02-02. Time integration
#### - Time: Explicit Euler
#### - Space: Central difference 

In [ ]:
#for l in range(1, nt-1):
#    for i in range(1, nx-1):
#        for j in range(1, ny-1):
#            #01. n
#            eta[1, i, j] = eta[0, i, j] - dt * (
#                                #01-01. rho * dui / dxi: Central difference
#                                rho0 * (
#                                    #01-01-01. du1 / dx
#                                    0.5 * (velx[0, i+1, j] - velx[0, i-1, j]) / dx
#
#                                    #01-01-02. du2 / dx2
#                                    + 0.5 * (vely[0, i, j+1] - vely[0, i, j-1]) / dy
#                                )
#
#                                #01-02. ui * dn / dxi: Central difference 
#                                + (
#                                    #01-02-01. u1 * dn/dx1
#                                    velx[0, i, j] * (0.5 * (eta[0, i+1, j] - eta[0, i-1, j]) / dx)
#
#                                    #01-02-02. u2 * dn/dx2
#                                    + vely[0, i, j] * (0.5 * (eta[0, i, j+1] - eta[0, i, j-1]) / dy)
#                                ) 
#
##                                #01-01. dui / dxi: Upwinding
##                                rho0 * (
##                                    #01-01-01. du1 / dx1
##                                    np.where(velx[0, i, j] > 0, 1., 0.) * (velx[0, i, j] - velx[0, i-1, j]) / dx
##                                    + np.where(velx[0, i, j] < 0, 1., 0.) * (velx[0, i+1, j] - velx[0, i, j]) / dx
##    
##                                    #01-01-02. du2 / dx2
##                                    + np.where(vely[0, i, j] > 0, 1., 0.) * (vely[0, i, j] - vely[0, i, j-1]) / dy
##                                    + np.where(vely[0, i, j] < 0, 1., 0.) * (vely[0, i, j+1] - vely[0, i, j]) / dy
##                                )
##
##                                #01-02. ui * dn / dxi: Upwinding 
##                                + (
##                                    #01-02-01. u1 * dn/dx1
##                                    velx[0, i, j] * (
##                                        np.where(velx[0, i, j] > 0, 1., 0.) * (eta[0, i, j] - eta[0, i-1, j]) / dx
##                                        + np.where(velx[0, i, j] < 0, 1., 0.) * (eta[0, i+1, j] - eta[0, i, j]) / dx
##                                    )
##
##                                    #01-02-02. u2 * dn/dx2
##                                    + vely[0, i, j] * (
##                                        np.where(vely[0, i, j] > 0, 1., 0.) * (eta[0, i, j] - eta[0, i, j-1]) / dy
##                                        + np.where(vely[0, i, j] < 0, 1., 0.) * (eta[0, i, j+1] - eta[0, i, j]) / dy
##                                    )
##                                )
#                            )
#            #02. u1    
#            velx[1, i, j] = velx[0, i, j] - dt * (
#                                #02-01. uj * du1 / dxj; Central difference
#                                #02-01-01. u1 * du1 / dx1
#                                velx[0, i, j] * (
#                                    0.5 * (velx[0, i+1, j] - velx[0, i-1, j]) / dx
#                                )
#                                
#                                #02-01-02. u2 * du1 / dx2
#                                + vely[0, i, j] * (
#                                    0.5 * (velx[0, i, j+1] - velx[0, i, j-1]) / dy
#                                )
#                                
#                                #02-02. g * dn / dx1; Central difference
#                                + g * (
#                                    0.5 * (eta[0, i+1, j] - eta[0, i-1, j]) / dx
#                                )
#
##                                #02-01. uj * du1 / dxj; Upwinding 
##                                #02-01-01. u1 * du1 / dx1
##                                velx[0, i, j] * (
##                                    np.where(velx[0, i, j] > 0, 1., 0.) * (velx[0, i, j] - velx[0, i-1, j]) / dx
##                                    + np.where(velx[0, i, j] < 0, 1., 0.) * (velx[0, i+1, j] - velx[0, i, j]) / dx
##                                )
##                                
##                                #02-01-02. u2 * du1 / dx2
##                                + vely[0, i, j] * (
##                                    np.where(vely[0, i, j] > 0, 1., 0.) * (velx[0, i, j] - velx[0, i, j-1]) / dy
##                                    + np.where(vely[0, i, j] < 0, 1., 0.) * (velx[0, i, j+1] - velx[0, i, j]) / dy
##                                )
##                                
##                                #02-02. g * dn / dx1; Upwinding
##                                + g * (
##                                    np.where(velx[0, i, j] > 0, 1., 0.) * (eta[0, i, j] - eta[0, i-1, j]) / dx
##                                    + np.where(velx[0, i, j] < 0, 1., 0.) * (eta[0, i+1, j] - eta[0, i, j]) / dx                                   
##                                )
#                            )
#            #03. u2    
#            vely[1, i, j] = vely[0, i, j] - dt * (
#                                #03-01. uj * du2 / dxj; Central difference
#                                #03-01-01. u1 * du2 / dx1
#                                velx[0, i, j] * 0.5 * (vely[0, i+1, j] - vely[0, i-1, j]) / dx
#                               
#                                #03-01-02. u2 * du2 / dx2
#                                + vely[0, i, j] * 0.5 * (vely[0, i, j+1] - vely[0, i, j-1]) / dy
#                               
#                                #03-02. g * dn / dx2; Central difference
#                                + g * 0.5 * (eta[0, i, j+1] - eta[0, i, j-1]) / dy
#
##                                #03-01. uj * du2 / dxj; Upwinding
##                                #03-01-01. u1 * du2 / dx1
##                                velx[0, i, j] * (
##                                    np.where(velx[0, i, j] > 0, 1., 0.) * (vely[0, i, j] - vely[0, i-1, j]) / dx
##                                    + np.where(velx[0, i, j] < 0, 1., 0.) * (vely[0, i+1, j] - vely[0, i, j]) / dx
##                                )
##                               
##                               #03-01-02. u2 * du2 / dx2
##                                + vely[0, i, j] * (
##                                    np.where(velx[0, i, j] > 0, 1., 0.) * (vely[0, i, j] - vely[0, i, j-1]) / dx
##                                    + np.where(vely[0, i, j] < 0, 1., 0.) * (vely[0, i, j+1] - vely[0, i, j]) / dy
##                                )
##                               
##                               #03-02. g * dn / dx2; Upwinding
##                                + g * (
##                                    np.where(vely[0, i, j] > 0, 1., 0.) * (eta[0, i, j] - eta[0, i-1, j]) / dy
##                                    + np.where(vely[0, i, j] < 0, 1., 0.) * (eta[0, i+1, j] - eta[0, i, j]) / dy                                   
##                                )
#                            )
#
#    # Apply bouncary condition (B.C.) 
#    # Neumann B.C. for n         
#    eta[1, 0, :] = eta[1, 1, :]
#    eta[1, nx-1, :] = eta[1, nx-2, :]
#    eta[1, :, 0] = eta[1, :, 1]
#    eta[1, :, ny-1] = eta[1, :, ny-2]
#   
#    # Dirichlet B.C. for u, v        
#    velx[1, 0, :] = 0.
#    velx[1, nx-1, :] = 0.
#    velx[1, :, 0] = 0.
#    velx[1, :, ny-1] = 0.
#   
#    vely[1, 0, :] = 0.
#    vely[1, nx-1, :] = 0.
#    vely[1, :, 0] = 0.
#    vely[1, :, ny-1] = 0.
#    
#    # Update (n)th timestep data with (n+1) timestep data 
#    # 0: (n)th timestep (Present)
#    # 1: (n+1)th timestep (Future)
#    eta[0, :, :] = copy.deepcopy(eta[1, :, :])
#    velx[0, :, :] = copy.deepcopy(velx[1, :, :])
#    vely[0, :, :] = copy.deepcopy(vely[1, :, :])
#
#    # Sampling/Visualizing the data
#    if(l % sample_freq) == 0:
#        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
#        surf = ax.plot_surface(X, Y, eta[0, :, :], cmap=cm.coolwarm,
#                                vmin=(1.0-0.08)*rho0, vmax=(1.0+0.08)*rho0, 
#                                linewidth=0, antialiased=False)
#
##        ax.view_init(elev=10,azim=10)
#
#        ax.set_xlim(x_min, x_max)
#        ax.set_ylim(y_min, y_max)
#        ax.set_zlim((1.0-0.08)*rho0, (1.0+0.08)*rho0)
#
#        formatter = ticker.ScalarFormatter(useMathText=True)
#
#        # Formatting of scientific representation
#        formatter.set_scientific(True) 
#
#        # Formatting of floating point representation
#        formatter.set_powerlimits((-2,2)) 
#        formatter.set_scientific
#        ax.zaxis.set_major_formatter(formatter)
#
##        # Make the panes transparent(1)
##        ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
##        ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
##        ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#
#        # Make the panes transparent(2)
#        ax.xaxis.pane.fill = False
#        ax.yaxis.pane.fill = False
#        ax.zaxis.pane.fill = False
#        # Set color to white
#        ax.xaxis.pane.set_edgecolor('w')
#        ax.yaxis.pane.set_edgecolor('w')
#        ax.zaxis.pane.set_edgecolor('w')
#
#        # Make the grid lines transparent(1)
##        ax.xaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)
##        ax.yaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)
##        ax.zaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)
#
#        # Make the grid lines transparent(2): get rid of grid
##        ax.grid(False)
#
#        # Add a color bar which maps values to colors.
#        fig.colorbar(surf, shrink=0.5, aspect=15, pad=0.15, extend='both')
#
#        # Set title
#        present_time = t_sta + l * dt 
#        ax.set_title('Elapsed Time: ' + str(present_time) + '[s]', loc='left')
#        ax.set_xlabel('x [m]')
#        ax.set_ylabel('y [m]')
#
#        # Save figure
#        #fname = 'central2d/img_finite_media/euler/continuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
#        fname = 'central2d/img_finite_media/euler/discontinuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
#        plt.savefig(fname, format='tiff', bbox_inches='tight', dpi=200)
#        id_snapshot += 1
#
#        plt.show()
#            


### 02-02. Time integration
#### - Time: Leapfrog integration**
#### - Space: Central difference 

###### **Unlike Euler integration, it is stable for oscillatory motion.

In [ ]:
for l in range(1, nt-1):

    # Predict velocity at (l+1/2) timestep: u1^(l+1/2) and u2^(l+1/2)
    for i in range(1, nx-1):
        for j in range(1, ny-1):

            #01. u1^(l+1/2)    
            velx[2, i, j] = velx[0, i, j] - 0.5 * dt * (
                                #01-01. uj * du1 / dxj; Central difference
                                #01-01-01. u1 * du1 / dx1
                                velx[0, i, j] * (
                                    0.5 * (velx[0, i+1, j] - velx[0, i-1, j]) / dx
                                )
                                
                                #01-01-02. u2 * du1 / dx2
                                + vely[0, i, j] * (
                                    0.5 * (velx[0, i, j+1] - velx[0, i, j-1]) / dy
                                )
                                
                                #01-02. g * dn / dx1; Central difference
                                + g * (
                                    0.5 * (eta[0, i+1, j] - eta[0, i-1, j]) / dx
                                )
                            )
            #02. u2^(l+1/2)    
            vely[2, i, j] = vely[0, i, j] - 0.5 * dt * (
                                #02-01. uj * du2 / dxj; Central difference
                                #02-01-01. u1 * du2 / dx1
                                velx[0, i, j] * 0.5 * (vely[0, i+1, j] - vely[0, i-1, j]) / dx
                               
                                #02-01-02. u2 * du2 / dx2
                                + vely[0, i, j] * 0.5 * (vely[0, i, j+1] - vely[0, i, j-1]) / dy
                               
                                #02-02. g * dn / dx2; Central difference
                                + g * 0.5 * (eta[0, i, j+1] - eta[0, i, j-1]) / dy
                            )

    # Apply bouncary condition (B.C.) 
    # Dirichlet B.C. for u, v        
    velx[2, 0, :] = 0.
    velx[2, nx-1, :] = 0.
    velx[2, :, 0] = 0.
    velx[2, :, ny-1] = 0.
   
    vely[2, 0, :] = 0.
    vely[2, nx-1, :] = 0.
    vely[2, :, 0] = 0.
    vely[2, :, ny-1] = 0.

    # Obtain n^(l+1) using u1^(l+1/2) and u2^(l+1/2)
    for i in range(1, nx-1):
        for j in range(1, ny-1):

            #03. n^(l+1)
            eta[1, i, j] = eta[0, i, j] - dt * (
                                #03-01. rho * dui / dxi: Central difference
                                rho0 * (
                                    #03-01-01. du1 / dx
                                    0.5 * (velx[2, i+1, j] - velx[2, i-1, j]) / dx

                                    #03-01-02. du2 / dx2
                                    + 0.5 * (vely[2, i, j+1] - vely[2, i, j-1]) / dy
                                )

                                #03-02. ui * dn / dxi: Central difference 
                                + (
                                    #03-02-01. u1 * dn/dx1
                                    velx[2, i, j] * (0.5 * (eta[0, i+1, j] - eta[0, i-1, j]) / dx)

                                    #03-02-02. u2 * dn/dx2
                                    + vely[2, i, j] * (0.5 * (eta[0, i, j+1] - eta[0, i, j-1]) / dy)
                                ) 
                            )

    # Apply bouncary condition (B.C.) 
    # Neumann B.C. for n         
    eta[1, 0, :] = eta[1, 1, :]
    eta[1, nx-1, :] = eta[1, nx-2, :]
    eta[1, :, 0] = eta[1, :, 1]
    eta[1, :, ny-1] = eta[1, :, ny-2]

    # Obtain velocity (u1, u2) using n^(l+1) and predited value of u1^(l+1/2) and u2^(l+1/2)
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            #02. u1    
            velx[1, i, j] = velx[2, i, j] - 0.5 * dt * (
                                #02-01. uj * du1 / dxj; Central difference
                                #02-01-01. u1 * du1 / dx1
                                velx[2, i, j] * (
                                    0.5 * (velx[2, i+1, j] - velx[2, i-1, j]) / dx
                                )
                                
                                #02-01-02. u2 * du1 / dx2
                                + vely[2, i, j] * (
                                    0.5 * (velx[2, i, j+1] - velx[2, i, j-1]) / dy
                                )
                                
                                #02-02. g * dn / dx1; Central difference
                                + g * (
                                    0.5 * (eta[1, i+1, j] - eta[1, i-1, j]) / dx
                                )
                            )
            #03. u2    
            vely[1, i, j] = vely[2, i, j] - 0.5 * dt * (
                                #03-01. uj * du2 / dxj; Central difference
                                #03-01-01. u1 * du2 / dx1
                                velx[2, i, j] * 0.5 * (vely[2, i+1, j] - vely[2, i-1, j]) / dx
                               
                                #03-01-02. u2 * du2 / dx2
                                + vely[2, i, j] * 0.5 * (vely[2, i, j+1] - vely[2, i, j-1]) / dy
                               
                                #03-02. g * dn / dx2; Central difference
                                + g * 0.5 * (eta[1, i, j+1] - eta[1, i, j-1]) / dy
                            )

    # Apply bouncary condition (B.C.) 
    # Dirichlet B.C. for u, v        
    velx[1, 0, :] = 0.
    velx[1, nx-1, :] = 0.
    velx[1, :, 0] = 0.
    velx[1, :, ny-1] = 0.
   
    vely[1, 0, :] = 0.
    vely[1, nx-1, :] = 0.
    vely[1, :, 0] = 0.
    vely[1, :, ny-1] = 0.
    
    # Update (l)th timestep data with (l+1) timestep data 
    # 0: (l)th timestep (Present)
    # 1: (l+1)th timestep (Future)
    eta[0, :, :] = copy.deepcopy(eta[1, :, :])
    velx[0, :, :] = copy.deepcopy(velx[1, :, :])
    vely[0, :, :] = copy.deepcopy(vely[1, :, :])

    # Sampling/Visualizing the data
    if(l % sample_freq) == 0:
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
        surf = ax.plot_surface(X, Y, eta[0, :, :], cmap=cm.coolwarm,
                                vmin=(1.0-0.08)*rho0, vmax=(1.0+0.08)*rho0, 
                                linewidth=0, antialiased=False)

#        ax.view_init(elev=10,azim=10)

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_zlim((1.0-0.08)*rho0, (1.0+0.08)*rho0)

        formatter = ticker.ScalarFormatter(useMathText=True)

        # Formatting of scientific representation
        formatter.set_scientific(True) 

        # Formatting of floating point representation
        formatter.set_powerlimits((-2,2)) 
        formatter.set_scientific
        ax.zaxis.set_major_formatter(formatter)

#        # Make the panes transparent(1)
#        ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#        ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#        ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

        # Make the panes transparent(2)
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        # Set color to white
        ax.xaxis.pane.set_edgecolor('w')
        ax.yaxis.pane.set_edgecolor('w')
        ax.zaxis.pane.set_edgecolor('w')

        # Make the grid lines transparent(1)
#        ax.xaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)
#        ax.yaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)
#        ax.zaxis._axinfo['grid']['color'] = (1.0, 1.0, 1.0, 0.0)

        # Make the grid lines transparent(2): get rid of grid
#        ax.grid(False)

        # Add a color bar which maps values to colors.
        fig.colorbar(surf, shrink=0.5, aspect=15, pad=0.15, extend='both')

        # Set title
        present_time = t_sta + l * dt 
        ax.set_title('Elapsed Time: ' + str(present_time) + '[s]', loc='left')
        ax.set_xlabel('x [m]')
        ax.set_ylabel('y [m]')

        # Save figure
        fname = 'central2d/img_finite_media/leapfrog/continuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
#        fname = 'central2d/img_finite_media/leapfrog/discontinuous_ic/ww2d_height_' + str(id_snapshot).zfill(5) + '.tiff'
        plt.savefig(fname, format='tiff', bbox_inches='tight', dpi=200)
        id_snapshot += 1

        plt.show()
            